then finish fine tuning using Lora
and do the classification again on all 3 datasets
oversample 2020 using smote


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [2]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/letters_noFT.json')
#df = df.drop("Predict", axis=1)
#df['emb_biogpt_no_FineT'] = df['emb_biogpt_no_FineT'].apply(lambda x: np.array(x.split(','), dtype=float))

# Create the StandardScaler object
scaler = StandardScaler()

In [3]:
df.columns

Index(['clinical_action', 'extracted_text', 'emb_biogpt_no_FineT',
       'emb_Stanford_no_FineT', 'emb_LLaMA_no_FineT'],
      dtype='object')

In [4]:
X_biogpt, y_biogpt = df.emb_biogpt_no_FineT.tolist(), df.clinical_action
X_stan, y_stan = df.emb_Stanford_no_FineT.tolist(), df.clinical_action
X_llama, y_llama = df.emb_LLaMA_no_FineT.tolist(), df.clinical_action

X_biogpt = np.squeeze(X_biogpt)
X_stan = np.squeeze(X_stan)
X_llama = np.squeeze(X_llama)

In [5]:
# Fit the scaler on the training data and transform it
X_biogpt = scaler.fit_transform(X_biogpt)
X_stan = scaler.fit_transform(X_stan)
X_llama = scaler.fit_transform(X_llama)

In [6]:
from sklearn.svm import SVC
#create SVC with class weight for predict column
svm_model_biogpt = SVC()
svm_model_stanford = SVC()
svm_model_llama = SVC()

In [7]:
def tunning(svc, X, y):
    param_grid = {'degree': [1, 2, 3],
                  'kernel': ['rbf', 'poly'],
                  'class_weight': ['balanced'],
                  'gamma':[0.01, 0.1, 1],
                  'C': [0.001, 0.1, 1]}
    grid = GridSearchCV(svc, param_grid, refit = True, verbose = 3, scoring= 'f1_macro', return_train_score=True, cv=5)

    # fitting the model for grid search
    grid.fit(X, y)

    # print best parameter after tuning
    best_params= grid.best_estimator_

    print(best_params)

    # print how our model looks after hyper-parameter tuning
    return best_params

In [8]:
#get best parameters for svc
svm_model_biogpt=tunning(svm_model_biogpt, X_biogpt, y_biogpt)
svm_model_stanford=tunning(svm_model_stanford, X_stan, y_stan)
svm_model_llama=tunning(svm_model_llama, X_llama, y_llama)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=rbf;, score=(train=0.432, test=0.429) total time=   0.0s
[CV 2/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=rbf;, score=(train=0.432, test=0.429) total time=   0.0s
[CV 3/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=rbf;, score=(train=0.432, test=0.429) total time=   0.0s
[CV 4/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=rbf;, score=(train=0.432, test=0.429) total time=   0.0s
[CV 5/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=rbf;, score=(train=0.814, test=0.912) total time=   0.0s
[CV 1/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.432, test=0.429) total time=   0.0s
[CV 2/5] END C=0.001, class_weight=balanced, degree=1, gamma=0.01, kernel=poly;, score=(train=0.432, test=0.429) total time=   0.1s
[CV 3/5] END C=0.00

In [9]:
from sklearn.model_selection import cross_val_predict

#predict
y_pred_biogpt = cross_val_predict(svm_model_biogpt, X_biogpt, y_biogpt, cv=5)
y_pred_stan = cross_val_predict(svm_model_stanford, X_stan, y_stan, cv=5)
y_pred_llama = cross_val_predict(svm_model_llama, X_llama, y_llama, cv=5)

In [10]:
from sklearn.metrics import classification_report
print("--------------------------Results letters---------------------------")
print()
print("-----------------------------Bio GPT--------------------------------")
print(classification_report(y_biogpt, y_pred_biogpt))
print("----------------------------stanford-------------------------------")
print(classification_report(y_stan, y_pred_stan))
print("-----------------------------LLaMA---------------------------------")
print(classification_report(y_llama, y_pred_llama))

--------------------------Results letters---------------------------

-----------------------------Bio GPT--------------------------------
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        75
           1       0.77      0.71      0.74        24

    accuracy                           0.88        99
   macro avg       0.84      0.82      0.83        99
weighted avg       0.88      0.88      0.88        99

----------------------------stanford-------------------------------
              precision    recall  f1-score   support

           0       0.89      0.91      0.90        75
           1       0.70      0.67      0.68        24

    accuracy                           0.85        99
   macro avg       0.80      0.79      0.79        99
weighted avg       0.85      0.85      0.85        99

-----------------------------LLaMA---------------------------------
              precision    recall  f1-score   support

           0    